# Scraper

In [1]:
from extract.common import credentials
from extract.common import config

In [ ]:
import pymongo
from bson.json_util import dumps

host = config()['mongodb']['host']
user = credentials()['mongodb']['user']
password = credentials()['mongodb']['password']

class SaveProjects(object):
    def __init__(self):
        uri = "mongodb+srv://{}:{}@{}/test?retryWrites=true".format(user,password,host)
        client = pymongo.MongoClient(uri)
        self._db = client[config()['mongodb']['db']['name']]
    
    def save(self, projects):
        self._db.project.insert_many(projects)

In [ ]:
save_project = SaveProjects()

In [ ]:
save_project._db.project.delete_many({})

In [ ]:
save_project._db.project.estimated_document_count()
[i for i in save_project._db.project.find()]

In [ ]:
save_project.save([{"a":1},{"b":2}])

In [2]:
from math import ceil
def year_to_periods(year, period_size):
    return ceil(365*year/period_size)
year_to_periods(4, 90)

17

In [3]:
from calculator.cumplo import predict_by_year
from calculator.cumplo import investment_return
from calculator.cumplo import fix_capital_no_extra
from calculator.cumplo import to_dataframe
from calculator.cumplo import predict_by_year_no_extra

In [20]:
vehicles = [{
    "time": 30,
    "yield": 0.18,
    "capital": 50000
} for i in range(84)]
vehicles[0]['capital'] = 0
prediction = predict_by_year_no_extra(vehicles)
prediction

{'time_serie': [{'time': 30,
   'extra_money': 0,
   'return': {'revenue': 0.0,
    'commission_percentage': 0.077328,
    'commission': 0.0,
    'return': 0.0},
   'capital': 0.0},
  {'time': 60,
   'extra_money': 0,
   'return': {'revenue': 750.0,
    'commission_percentage': 0.077328,
    'commission': 57.995999999999995,
    'return': 692.004},
   'capital': 50692.004},
  {'time': 90,
   'extra_money': 0,
   'return': {'revenue': 1500.0,
    'commission_percentage': 0.077328,
    'commission': 115.99199999999999,
    'return': 1384.008},
   'capital': 102076.012},
  {'time': 120,
   'extra_money': 0,
   'return': {'revenue': 2280.0,
    'commission_percentage': 0.077328,
    'commission': 176.30784,
    'return': 2103.69216},
   'capital': 154179.70416},
  {'time': 150,
   'extra_money': 0,
   'return': {'revenue': 3060.0,
    'commission_percentage': 0.077328,
    'commission': 236.62367999999998,
    'return': 2823.37632},
   'capital': 207003.08048},
  {'time': 180,
   'extra_mo

In [21]:
import iplotter
from IPython.core.display import HTML
from visualize.chart import Chart

In [22]:
df = to_dataframe(prediction)
invested = (df['capital']-df['return']).tolist()
profits = df['return'].tolist()
x = df.index.strftime('%Y-%m-%d').tolist()

In [19]:
chart = Chart().stacked_area(x, profits, invested)
HTML(Chart.html)
c3_plotter = iplotter.C3Plotter()
c3_plotter.plot(chart)